# Plot functions

[Here we put some plot functions that we use more often]

## General plot options

Some plot functions that are used by other functions

In [1]:
# changes the general size of the plots
size_multiplier = 1

# function makes folder, pdf, png and web png (small)
def save_plot(title):
    if not os.path.exists(plot_folder):
        os.makedirs(plot_folder)
    if not os.path.exists(plot_folder_web):
        os.makedirs(plot_folder_web)
    if save_pdf_plots:
        plt.savefig(os.path.join(plot_folder, title) + '.pdf', format='pdf')
    if save_plots:
        plt.savefig(os.path.join(plot_folder, title) + '.png', format='png', dpi=300)
        plt.savefig(os.path.join(plot_folder_web, title) + '.png', format='png', dpi=50)

# function to set the axis
def set_axis(xaxis,yaxis,title):
    plt.xlabel(xaxis, fontsize = 16)
    plt.ylabel(yaxis, fontsize = 16)
    plt.title(title, fontsize = 24, y =1.03)
    plt.tick_params(axis='both', which='major', labelsize=16)
    plt.tick_params(axis='both', which='minor', labelsize=16)

## Scatter plots

In [2]:
# actual scatter plotting function that uses set_axis and save_plot
def plot_scatter(x,y,xtitle,ytitle,title,xlim=[None,None],ylim=[None,None], colorb = False, 
                 colorblabel=None, discrete = False, vmax = None, *args, **kwargs):
    if discrete:
        plt.scatter(x,y,vmax = vmax,cmap = plt.get_cmap('jet', vmax), *args, **kwargs)
    else:
        plt.scatter(x,y,vmax = vmax, *args, **kwargs)
    plt.xlim(xlim)
    plt.ylim(ylim)
    set_axis(xtitle,ytitle,title)
    if colorb:
        if discrete == True:
            tick_bounds = np.linspace(0,vmax,vmax+1)
            tickpos = np.linspace(tick_bounds[0] + .5, tick_bounds[-1] - .5, len(tick_bounds)-1)
            cax = plt.colorbar(label=colorblabel, ticks=tickpos)
            cax.set_ticklabels(tick_bounds)
        else:
            plt.colorbar(label=colorblabel)
    save_plot(title)
    plt.grid()
    plt.show()
 


## Histograms

In [3]:
# function for getting bin centers for histograms
def bin_centers(bin_edges):
    return 0.5*(bin_edges[1:] + bin_edges[:-1])

# function for 2D histogram
def twod_stat_plot(x, y, z, statistic='mean', bins=10, range=None, vrange=None, cblabel=None):
    if vrange is None:
        vrange = (None, None)
    result, xbinedges, ybinedges, binnumbers = binned_statistic_2d(
        x, y, z,
        bins=bins,
        statistic=statistic,
        range=range
    )
    xx, yy = np.meshgrid(xbinedges, ybinedges)    

    Zm = np.ma.masked_where(np.isnan(result),result)
    plt.figure(figsize=(size_multiplier*12, size_multiplier*10))
    plt.pcolormesh(xx, yy, Zm.T, vmin=vrange[0], vmax=vrange[1])
    plt.colorbar(label=cblabel)
    
# Function to make the colorbar in discrete steps
def discrete_matshow(data):
    cmap = plt.get_cmap('RdBu', np.max(data)-np.min(data)+1)
    mat = plt.matshow(data,cmap=cmap)
    ticks = np.arange(np.min(data),np.max(data)+1)
    tickpos = np.linspace(ticks[0] + .5, ticks[-1] - .5, len(ticks));
    cax = plt.colorbar(mat, ticks=tickpos)
    cax.set_ticklabels(ticks)

### Using a more powerful hist package 

This uses Jelle's multihist package

In [4]:
from multihist import *

def plot_2dhist(x,y,xtitle,ytitle,title,rangexy = [[None,None],[None,None]], colorb = False, 
                 colorblabel=None, bins =[100,100],vmin=None, vmax = None, *args, **kwargs):
    m2 = Histdd(bins=bins, range=rangexy,*args, **kwargs)
    m2.add(x,y)
    m2.plot(cblabel = colorblabel, vmin=vmin, vmax=vmax)
    set_axis(xtitle,ytitle,title)
    plt.show()

## For plotting PMT arrays

In [5]:
## For PMT arrays
def plot_pmts(pmt_range,  s=20, colorbar =True, **kwargs):
    plt.scatter([sim.config['pmt_locations'][i]['x'] for i in pmt_range], 
                [sim.config['pmt_locations'][i]['y'] for i in pmt_range],
                 s=s, edgecolors='white', **kwargs)
    if colorbar:
        plt.colorbar(label = 'S2 light (pe)')
    
def plot_top_array_dead():
    plot_pmts([ch for ch in sim.config['channels_top'] if ch in dead_pmts and ch != 0],
              s=40, c='red', marker = 's', colorbar =False)
    
def plot_bottom_array_dead():
    plot_pmts([ch for ch in sim.config['channels_bottom'] if ch in dead_pmts and ch != 0],
              s=40, c='red', marker = 's', colorbar =False)

def plot_top_saturated(saturated_channel_numbers):
    plot_pmts([ch for ch in saturated_channel_numbers if ch in sim.config['channels_top']],
               s = 20, c = 'black', marker = 'D', colorbar =False)
    
def plot_bottom_saturated(saturated_channel_numbers):
    plot_pmts([ch for ch in saturated_channel_numbers if ch in sim.config['channels_bottom']],
               s = 20, c = 'black', marker = 'D', colorbar =False)

def plot_top_array(light,size_multiplier, **kwargs):
    plot_pmts([ch for ch in sim.config['channels_top'] if ch not in dead_pmts],
              s=np.absolute(light)*size_multiplier, c=light ,marker = 'o', **kwargs)
    
def plot_bottom_array(light,size_multiplier, **kwargs):
    plot_pmts([ch for ch in sim.config['channels_bottom'] if ch not in dead_pmts],
              s=np.absolute(light)*size_multiplier, c=light ,marker = 'o', **kwargs)

def plot_detector_radius():
    plt.xlim((-1.1*detector_radius, 1.1*detector_radius))
    plt.ylim((-1.1*detector_radius, 1.1*detector_radius))

    plt.gca().add_artist(plt.Circle((0,0), 
                                    detector_radius, 
                                    edgecolor='black', 
                                    fill=None))
    plt.gca().add_artist(plt.Circle((0,0), 
                                    fiducial_volume_radius, 
                                    edgecolor='0.7',
                                    fill=None))

### How to change font in plots

Just some instructions on how to change the font of the text in your plots

In [6]:
# 1. Install arial on your system
# sudo apt-get install fonts-liberation
# sudo apt-get install msttcorefonts
# sudo fc-cache -f -v

# 2. delete your matplotlib cash in ~/.matplotlib or ~/.cache/matplotlib

# 3. In your code import matplotlib and set font to Arial
# (import matplotlib.pyplot as plt)

# plt.rcParams.update({'font.sans-serif':'Arial'})

# To check:
# fc-list | grep -i "Arial"

# Cheers, you now have another font for publishing in papers